<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="75" width="75" align="left"/>

# Usage of OpenEnergyPlatform API-Dialect (oedialect)

Repository: https://github.com/openego/oedialect <br>
Documentation: http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/oedialect/issues <br>

In [1]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "henhuy, Ludee, oakca"

In [2]:
# pip install oedialect pandas "sqlalchemy~=1.3,<1.4"

import os
import getpass

import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import BigInteger, Boolean, Column, Date, Float, Integer, Numeric, String, Table, Text, text
from geoalchemy2.types import Geometry
from sqlalchemy.ext.declarative import declarative_base
import oedialect


## Connection to OEP

If we want to upload data to the OEP we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token on your OEP profile page after logging in.

In [3]:
# NEVER commit your token to a repository
# get your token from an environment variable
# or ask user
token = os.environ.get("OEP_API_TOKEN") or getpass.getpass('Token:')

In [4]:
# Create Engine:
oep_host = 'openenergyplatform.org'
oed_string = f'postgresql+oedialect://:{token}@{oep_host}'

engine = sa.create_engine(oed_string)
metadata = sa.MetaData(bind=engine)

## Setup a Table

We need to tell the data base what columns and datatypes we are about to upload. In our case we have four columns, two of which are text, one is integer and the last is float.

In [5]:
tname = 'example_dialect_table'
sname = 'sandbox'

table = sa.Table(
    tname,
    metadata,
    sa.Column('name', sa.VARCHAR(50)),
    sa.Column('age', sa.INTEGER),
    schema=sname
)

## Create the new Table

Now we tell our engine to connect to the data base and create the defined table within the chosen schema.

In [6]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, tname, sname):
    table.create()
    print('Created table')

Connection established


## Insert data into Table
 
Uploading the information from our DataFrame is now done with a single command. Uploading data in this way will always delete the content of the table and refill it with new values every time. If you change 'replace' to 'append', the data entries will be added to the preexisting ones. (Connecting and uploading may take a minute.)

In [7]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = table.insert().values(
        [
            dict(name='Peter', age=25),
            dict(name='Inge', age=42),
            dict(name='Horst', age=36)
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    raise
finally:
    session.close()

Insert successful!


## Select from Table

Now  we can query our table to see if the data arrived.

In [8]:
# ORM classes -> ego.io

Base = declarative_base()
metadata = Base.metadata

class BkgVg2502Lan(Base):
    __tablename__ = 'bkg_vg250_2_lan'
    __table_args__ = {'schema': 'boundaries'}

    reference_date = Column(Date, primary_key=True, nullable=False)
    id = Column(Integer, primary_key=True, nullable=False, server_default=text("nextval('boundaries.bkg_vg250_2_lan_id_seq'::regclass)"))
    ade = Column(Float(53))
    gf = Column(Float(53))
    bsg = Column(Float(53))
    rs = Column(String(12))
    ags = Column(String(12))
    sdv_rs = Column(String(12))
    gen = Column(String(50))
    bez = Column(String(50))
    ibz = Column(Float(53))
    bem = Column(String(75))
    nbd = Column(String(4))
    sn_l = Column(String(2))
    sn_r = Column(String(1))
    sn_k = Column(String(2))
    sn_v1 = Column(String(2))
    sn_v2 = Column(String(2))
    sn_g = Column(String(3))
    fk_s3 = Column(String(2))
    nuts = Column(String(5))
    rs_0 = Column(String(12))
    ags_0 = Column(String(12))
    wsk = Column(Date)
    debkg_id = Column(String(16))
geom = Column(Geometry('MULTIPOLYGON', 31467), index=True)

In [9]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(BkgVg2502Lan).all())
session.close()

[<__main__.BkgVg2502Lan object at 0x00000280AA591CC0>, <__main__.BkgVg2502Lan object at 0x00000280AA591BA0>, <__main__.BkgVg2502Lan object at 0x00000280AA5918A0>, <__main__.BkgVg2502Lan object at 0x00000280AA591570>, <__main__.BkgVg2502Lan object at 0x00000280AA591330>, <__main__.BkgVg2502Lan object at 0x00000280AA591450>, <__main__.BkgVg2502Lan object at 0x00000280AA5919C0>, <__main__.BkgVg2502Lan object at 0x00000280AA591BD0>, <__main__.BkgVg2502Lan object at 0x00000280AA5919F0>, <__main__.BkgVg2502Lan object at 0x00000280AA5918D0>, <__main__.BkgVg2502Lan object at 0x00000280AA591C90>, <__main__.BkgVg2502Lan object at 0x00000280AA591C30>, <__main__.BkgVg2502Lan object at 0x00000280AA591750>, <__main__.BkgVg2502Lan object at 0x00000280AA591D80>, <__main__.BkgVg2502Lan object at 0x00000280AA591720>, <__main__.BkgVg2502Lan object at 0x00000280AA591540>, <__main__.BkgVg2502Lan object at 0x00000280AA591780>, <__main__.BkgVg2502Lan object at 0x00000280AA591AE0>, <__main__.BkgVg2502Lan obje

In [10]:
df = pd.DataFrame(session.query(BkgVg2502Lan.id,BkgVg2502Lan.gen, BkgVg2502Lan.reference_date)
                  .filter(BkgVg2502Lan.reference_date == '2016-01-01').all())
df

,id,gen,reference_date
0,1,Saarland,2016-01-01
1,2,Baden-Württemberg,2016-01-01
2,3,Nordrhein-Westfalen,2016-01-01
3,5,Bremen,2016-01-01
4,6,Hessen,2016-01-01
5,7,Brandenburg,2016-01-01
6,8,Niedersachsen,2016-01-01
7,9,Mecklenburg-Vorpommern,2016-01-01
8,10,Rheinland-Pfalz,2016-01-01
9,11,Berlin,2016-01-01
